In [4]:
!pip -q install langchain_experimental langchain_core

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.12.1 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.2 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.2 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.1.0 which is incompatible.
sagema

In [5]:
import pandas as pd
import boto3
from io import BytesIO

s3_bucket = 'filesfornotebook'
s3_key = 'Facebook_data_V3.csv'

# Create an S3 client
s3_client = boto3.client('s3')

response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
csv_content = response['Body'].read()

# Try different encodings
try:
    df = pd.read_csv(BytesIO(csv_content), encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(BytesIO(csv_content), encoding='latin1')


In [6]:
df.head(2)

,Ad Set Name,Treatment,Location,Ad name,Result type,Results,Reach,Impressions,Cost per result,Amount spent (INR),Clicks (all),CTR (all) (click through rate),CPC (All) (cost per click),Reporting starts,Reporting ends
0,Lipoma-Coimbatore-Sep23,Lipoma,Coimbatore,Lipoma-Coimbatore-Video,Meta Leads,70,235498,482468,624.206857,43694.48,2747,0.569364,15.906254,9/1/2023,9/30/2023
1,Lipoma-Chennai-Sep23,Lipoma,Chennai,Lipoma-Chennai-Video,Meta Leads,80,247162,446255,412.803500,33024.28,1760,0.394393,18.763795,9/1/2023,9/30/2023


In [7]:
!pip install replicate

  Using cached replicate-0.22.0-py3-none-any.whl.metadata (22 kB)
  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
Using cached replicate-0.22.0-py3-none-any.whl (35 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requ

In [8]:
pip install replicate

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.llms import Replicate

In [11]:
import os

api_token = 'r8_d9o57OjPAfzTbBbFfCetgUSwQU2MjLO1Oxulv'

# Set the API token as an environment variable
os.environ['REPLICATE_API_TOKEN'] = api_token

In [12]:
import pandas as pd
import chardet
import boto3
from io import BytesIO, StringIO
from langchain_experimental.agents import create_csv_agent


# Specify the S3 bucket and key (file path) of your CSV file
s3_bucket = 'filesfornotebook'
s3_key = 'Facebook_data_V3.csv'

# Create an S3 client
s3_client = boto3.client('s3')

# Download the CSV file from S3
response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
csv_content = response['Body'].read()

# Detect the encoding using chardet
result = chardet.detect(csv_content)
encoding = result['encoding']
confidence = result['confidence']
print(f"Detected encoding: {encoding} with confidence {confidence}")

# Try reading the CSV file with the detected encoding
try:
    df = pd.read_csv(BytesIO(csv_content), encoding=encoding)
except UnicodeDecodeError:
    print(f"Reading with {encoding} encoding failed")
    

# Create an agent with the DataFrame
#agent = create_csv_agent(run_mistral_model, StringIO(df.to_csv(index=False)), verbose=True)

llm = Replicate(
    model="meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
  input={
    "debug": False,
    "top_k": 50,
    "top_p": 1,
    "temperature": 0.9,
    "max_new_tokens": 500,
    "min_new_tokens": -1
  }
)


Init param `input` is deprecated, please use `model_kwargs` instead.


Detected encoding: utf-8 with confidence 0.7525


In [13]:
agent = create_csv_agent(llm, StringIO(df.to_csv(index=False)), verbose=True)

In [14]:
agent

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': "|    | Ad Set Name              | Treatment    | Location   | Ad name                                     | Result type   |   Results |   Reach |   Impressions |   Cost per result |   Amount spent (INR) |   Clicks (all) |   CTR (all) (click through rate) |   CPC (All) (cost per click) | Reporting starts   | Reporting ends   |\n|---:|:-------------------------|:-------------|:-----------|:--------------------------------------------|:--------------|----------:|--------:|--------------:|------------------:|---------------------:|---------------:|---------------------------------:|-----------------------------:|:-------------------|:-----------------|\n|  0 | Lipoma-Coimbatore-Sep23  | Lipoma       | Coimbatore | Lipoma-Coimbatore-Video                     | Meta Leads    |        70 |  235498 |        482468 |           624

In [7]:
#agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [15]:
agent.run("What is the average and sum of Cost per result")



> Entering new AgentExecutor chain...
 Thought: Let's calculate the average and sum of Cost per result for the given dataset.

Action: python_repl_ast

Action Input: `df['Cost per result']`

Observation: 0      624.206857
1      412.803500
2      297.510833
3      462.789318
4      277.308451
          ...    
311      0.000000
312      0.000000
313      0.000000
314      0.000000
315      0.000000
Name: Cost per result, Length: 316, dtype: float64
Thought: Sure, I can help you with that!

To calculate the average and sum of Cost per result, we can use the `mean()` and `sum()` functions in pandas.

Action: python_repl_ast

Action Input: `df['Cost per result'].mean()`

Observation: 297.6987766160443
Thought: Great, now we know the mean of Cost per result is 297.6987766160443. Let's calculate the sum of Cost per result.

Action: python_repl_ast

Action Input: `df['Cost per result'].sum()`

Observation: 94072.81341067
Thought: Thought: Now that we have the sum of Cost per result, we can

ReplicateError: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

In [16]:
agent.run("what is the average and sum of Reach?")



> Entering new AgentExecutor chain...


ReplicateError: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

In [11]:
agent.run("Give me the top 5 rows of information from the data")



> Entering new AgentExecutor chain...


ReplicateError: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

In [ ]:
agent.run("Which ad set had the highest number of results? give me that ad set name")

In [ ]:
agent.run("What is the overall click-through rate (CTR) for the campaign?")

In [ ]:
agent.run("What is the overall click-through rate (CTR) for the campaign?")

In [ ]:
agent.run("Identify the ad set with the lowest cost per result.")


In [ ]:
agent.run("Compare the performance of 'Lipoma-Coimbatore-Sep23' and 'Lipoma-Chennai-Sep23' ad sets.")


In [ ]:
agent.run("Compare the performance of 'Lipoma-Coimbatore-Sep23' and 'Lipoma-Chennai-Sep23' ad sets.")


In [ ]:
agent.run("Which location performed the best in terms of results?")


In [ ]:
agent.run("How did the performance vary over the reporting period?")


In [ ]:
agent.run("Were there any specific dates with a significant increase or decrease in results?")


In [ ]:
agent.run("Compare the performance of different locations across all ad sets. and provide me response. show me the response in charts format")
